<a href="https://colab.research.google.com/github/LeonardoSanBenitez/open-capivara-explainer/blob/main/notebooks/5.0%20-%20Finetuning%20with%20LoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Process:
1. Set .env file with `HF_TOKEN=""`
1. Run the installation cell "Training dependencies setup"
2. Restart
3. Run all


In the guff conversion it says it needs restart agian, but worked without (I kept the guff setup separately because of dependency conflicts)

The downloaded models have to be put in the right folder afterwards, and maning adjusted

# References

[original tutorial](https://rocm.blogs.amd.com/artificial-intelligence/llama2-lora/README.html)

[colorist tutorial](https://mychen76.medium.com/tinyllama-colorist-fine-tuned-with-color-dataset-8cd1cf7e5665)

In [1]:
# Training dependencies setup
!pip install -q pandas peft==0.9.0 transformers==4.31.0 trl==0.4.7 bitsandbytes==0.43.0 accelerate scipy  tensorboardX python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

In [1]:
batch_size = 2
number_of_checkpoints = 4


# mlabonne/guanaco-llama2-1k is a subset (1,000 samples) of the timdettmers/openassistant-guanaco
# human-generated, human-annotated, assistant-style conversation corpus
#dataset_name = "mlabonne/guanaco-llama2-1k"

dataset_name = 'LeonardoBenitez/capivara-plugin-orchestration'
dataset_config = 'plugins_capital_v1.0'


base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
merged_model_repository = 'LeonardoBenitez/capivara-plugin-orchestration'
merged_model_config = f"{base_model_name.split('/')[1]}-LoRa-{dataset_config}"  # Becomes a folder in the repository
merged_model_name="merged_quantized.q8_0.gguf"  # Becomes a file inside that folder


# Training

In [48]:
import os
import subprocess
import sys
import math
import dotenv
from typing import Tuple
import torch
from huggingface_hub import HfApi
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
from peft import get_peft_model
from google.colab import files

api = HfApi()
dotenv.load_dotenv()

assert type(os.getenv('HF_TOKEN')) == str
assert len(os.getenv('HF_TOKEN')) > 0
assert torch.cuda.is_available()
!nvidia-smi

In [51]:
def exec_command(command: str) -> Tuple[bool, str]:
    try:
        output = subprocess.check_output(command, stderr=subprocess.STDOUT, shell=True, encoding=sys.getfilesystemencoding())
        worked = True
    except subprocess.CalledProcessError as e:
        output = f'{e.returncode} {str(e.output)}'
        worked = False
    return worked, output

In [3]:
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="cuda",
    #load_in_8bit=True,
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1
print(type(base_model))
base_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.ro

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Data set
training_data = load_dataset(dataset_name, dataset_config, split='train')
last_step = math.ceil(training_data.shape[0]/batch_size)
save_steps = int(training_data.shape[0]/number_of_checkpoints)  # Save a checkpoint every {save_steps} steps
print(training_data.shape)
print(training_data[0])

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

(4000, 1)
{'text': '<|system|>\nYou are allowed to call the following function:\n1. PluginCapital_get_capital: Returns the name of the capital of a country..\n  Arguments: country (Name of the country. Type: string. Required: yes)\n2. answer: Send response back to the user. Show all your results here, this is the only thing that the user will see. You won\'t be able to call any other function after this one. Be sure to return a complete and clear answer, the user will not be able to see any other intermediate messages nor ask for more information. Never mention intermediate messages or results; if you want to mention something, include it here. Call this function only once, with everything you want to show to the user..\n  Arguments: text (final textual response to be send to the user. Should use HTML syntax for formatting. Type: string. Required: yes)\n</s>\n\n<|user|>\nI need help with a geography exam about Albania, can you tell me what is its capital?</s>\n\n<|assistant|>\n{"though

In [5]:
if os.path.exists("/content/model_modified"):
  !rm -rf "/content/model_modified"
os.mkdir("/content/model_modified")

# Training Params
train_params = TrainingArguments(
    output_dir="/content/model_modified",
    num_train_epochs=1,  # tinyagent used 3 epochs
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=save_steps,
    logging_steps=50,
    learning_rate=1e-4,  # colorist used 2e-4; tinyagent used 7e-5; original tutorial used 4e-5
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",  # colorist used cosine
    report_to="tensorboard"
)

# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, peft_parameters)
model.print_trainable_parameters()

# Trainer with LoRA configuration
device = torch.device("cuda:0")
base_model= base_model.to(device)
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,
    max_seq_length=1024,  # colorist used this... but should not be the contex size, which is 2048?

)

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.10229075496156657


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [7]:
%%time
# Training
# will run for {last_step} steps
# May take a long time
# 4k rows, batch size 2, 1 epoch: 26 minutes
fine_tuning.train()

Step,Training Loss
50,0.166100
100,0.129800
150,0.111100
200,0.104400
250,0.085800
300,0.082000
350,0.073400
400,0.077600
450,0.066900
500,0.066200


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


CPU times: user 16min 1s, sys: 9min 22s, total: 25min 23s
Wall time: 25min 48s


TrainOutput(global_step=2000, training_loss=0.06942988586425781, metrics={'train_runtime': 1545.5515, 'train_samples_per_second': 2.588, 'train_steps_per_second': 1.294, 'total_flos': 7364274118483968.0, 'train_loss': 0.06942988586425781, 'epoch': 1.0})

In [ ]:
# Download model
# ~17MB
'''
assert os.path.exists(f"/content/model_modified/checkpoint-{last_step}")
os.system(f"zip -r model_modified_last_checkpoint.zip /content/model_modified/checkpoint-{last_step}")
assert os.path.exists("model_modified_last_checkpoint.zip")
files.download('model_modified_last_checkpoint.zip')
''';

  adding: content/model_modified/checkpoint-1000/ (stored 0%)
  adding: content/model_modified/checkpoint-1000/trainer_state.json (deflated 82%)
  adding: content/model_modified/checkpoint-1000/special_tokens_map.json (deflated 73%)
  adding: content/model_modified/checkpoint-1000/scheduler.pt (deflated 57%)
  adding: content/model_modified/checkpoint-1000/adapter_model.safetensors (deflated 8%)
  adding: content/model_modified/checkpoint-1000/tokenizer.model (deflated 55%)
  adding: content/model_modified/checkpoint-1000/training_args.bin (deflated 50%)
  adding: content/model_modified/checkpoint-1000/adapter_config.json (deflated 51%)
  adding: content/model_modified/checkpoint-1000/tokenizer_config.json (deflated 68%)
  adding: content/model_modified/checkpoint-1000/rng_state.pth (deflated 25%)
  adding: content/model_modified/checkpoint-1000/tokenizer.json (deflated 74%)
  adding: content/model_modified/checkpoint-1000/adapter_model.bin (deflated 8%)
  adding: content/model_modifie

# Test Inference

In [8]:
from peft import PeftModel
import os
from google.colab import files
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

assert torch.cuda.is_available()

In [9]:
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="cuda",
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Merged
inference_model = PeftModel.from_pretrained(base_model, f"/content/model_modified/checkpoint-{last_step}")
device = torch.device("cuda:0")
inference_model= inference_model.to(device)
print(type(inference_model))
inference_model

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.ro

In [39]:
batch = llama_tokenizer('''<|system|>
You are allowed to call the following function:
1. PluginCapital_get_capital: Returns the name of the capital of a country..
  Arguments: country (Name of the country. Type: string. Required: yes)
2. answer: Send response back to the user. Show all your results here, this is the only thing that the user will see. You won't be able to call any other function after this one. Be sure to return a complete and clear answer, the user will not be able to see any other intermediate messages nor ask for more information. Never mention intermediate messages or results; if you want to mention something, include it here. Call this function only once, with everything you want to show to the user..
  Arguments: text (final textual response to be send to the user. Should use HTML syntax for formatting. Type: string. Required: yes)
</s>

<|user|>
What is the capital of France?</s>

<|assistant|>''', return_tensors="pt")
batch = {k: v.to("cuda") for k, v in batch.items()}
output_tokens = inference_model.generate(
    **batch,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    top_k=50,
    eos_token_id=llama_tokenizer.eos_token_id,
    pad_token_id=llama_tokenizer.pad_token_id,
)
assert type(output_tokens) == torch.Tensor

assert len(output_tokens) == 1
assert len(output_tokens[0]) > 10
output_tokens

tensor([[    1,   529, 29989,  5205, 29989, 29958,    13,  3492,   526,  6068,
           304,  1246,   278,  1494,   740, 29901,    13, 29896, 29889,  1858,
          3851, 12415,  2410, 29918,   657, 29918,  5030,  2410, 29901, 16969,
           278,  1024,   310,   278,  7483,   310,   263,  4234,   636,    13,
         29871, 11842,  9331, 29901,  4234,   313,  1170,   310,   278,  4234,
         29889,  5167, 29901,  1347, 29889,   830,  5958, 29901,  4874, 29897,
            13, 29906, 29889,  1234, 29901, 15076,  2933,  1250,   304,   278,
          1404, 29889,  7704,   599,   596,  2582,  1244, 29892,   445,   338,
           278,   871,  2655,   393,   278,  1404,   674,  1074, 29889,   887,
          2113, 29915, 29873,   367,  2221,   304,  1246,   738,   916,   740,
          1156,   445,   697, 29889,  1522,  1854,   304,   736,   263,  4866,
           322,  2821,  1234, 29892,   278,  1404,   674,   451,   367,  2221,
           304,  1074,   738,   916, 19697,  7191,  

In [40]:
target_predicted = llama_tokenizer.decode(output_tokens[0], skip_special_tokens=False).split("<|endcontext|>")[0]
assert type(target_predicted) == str
print(target_predicted)

'<s> <|system|>\nYou are allowed to call the following function:\n1. PluginCapital_get_capital: Returns the name of the capital of a country..\n  Arguments: country (Name of the country. Type: string. Required: yes)\n2. answer: Send response back to the user. Show all your results here, this is the only thing that the user will see. You won\'t be able to call any other function after this one. Be sure to return a complete and clear answer, the user will not be able to see any other intermediate messages nor ask for more information. Never mention intermediate messages or results; if you want to mention something, include it here. Call this function only once, with everything you want to show to the user..\n  Arguments: text (final textual response to be send to the user. Should use HTML syntax for formatting. Type: string. Required: yes)\n</s> \n\n<|user|>\nWhat is the capital of France?</s> \n\n<|assistant|>\n{"thought": "I need to know the capital of france", "action_name": "PluginCa

# Quantization and GUFF conversion

Conversion instructions: https://github.com/ggerganov/llama.cpp/discussions/2948

Loading a GGUF model with python transformers: https://github.com/ggerganov/llama.cpp/discussions/2948

In [55]:
# Setup
import locale
locale.getpreferredencoding = "UTF-8"
if os.path.exists("llama.cpp"):
  exec_command('rm -rf "llama.cpp"')
os.mkdir("llama.cpp")

worked, output = exec_command('git clone "https://github.com/ggerganov/llama.cpp.git"')
assert worked
worked, output = exec_command('pip install -r llama.cpp/requirements.txt')
assert worked
#!python llama.cpp/convert_lora_to_gguf.py -h
#!ls -l llama.cpp/convert*

In [42]:
# Saving the lora weights + base model (1~2 GBs)
# You can run the inference with both `inference_model` or `merged_model`
# Since the merge is oly needed for saving, i'm doing it here
merged_model = inference_model.merge_and_unload()
merged_model.save_pretrained("/content/model_merged", save_adapters=True, save_embedding_layers=True)
llama_tokenizer.save_pretrained("/content/model_merged")

worked, output = exec_command('python llama.cpp/convert_hf_to_gguf.py "/content/model_merged" --outfile "/content/model_modified_merged_quantized.gguf" --outtype q8_0')
assert worked
assert os.path.exists("/content/model_modified_merged_quantized.gguf")

# Upload the resulting file to HuggingFace
# To use this file with LlamaCPP: ["--hf-repo", merged_model_repository, "--hf-file", "{merged_model_config}/{merged_model_name}""]
api.upload_file(
    path_or_fileobj="/content/model_modified_merged_quantized.gguf",  # Path to the model file
    path_in_repo=os.path.join(merged_model_config, merged_model_name),
    repo_id=merged_model_repository,
    repo_type="model",
)

# Or you can download the file directly
# This finish executing before the model is fully downloaded, so wait...
# To use this file with llamaCPP: command: ["--model", "/models/model_modified_merged_quantized.gguf"]
'''
files.download('/content/model_modified_merged_quantized.gguf')
''';

# Zipping reduce about 4% size, so whatever...
'''
!zip /content/model_modified_merged_quantized.gguf.zip /content/model_modified_merged_quantized.gguf
!ls -lah /content/model_modified_merged_quantized.gguf.zip
files.download('/content/model_modified_merged_quantized.gguf.zip')
''';


# Another option is saving to your personal drive
# This is faster and more reliable, but you have to setup the drive connection
'''
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
!cp "/content/model_modified_merged_quantized.gguf" "/content/gdrive/MyDrive/PATH_IN_YOUR_DRIVE/model_modified_merged_quantized.gguf"
''';

# Maybe `gdown` is also an option
# https://stackoverflow.com/questions/49428332/how-to-download-large-files-like-weights-of-a-model-from-colaboratory

model_modified_merged_quantized.gguf:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

In [ ]:
# Saving base model
'''
from huggingface_hub import snapshot_download
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
snapshot_download(
    repo_id=model_id,
    local_dir="/content/model_base",
    local_dir_use_symlinks=False,
    revision="main",
  )

!python llama.cpp/convert_hf_to_gguf.py "/content/model_base" \
  --outfile "/content/model_base.gguf" \
  --outtype q8_0

assert os.path.exists("/content/model_base.gguf")
files.download('/content/model_base.gguf')
''';

# Saving just the lora weights (a few MBs)
'''
!python llama.cpp/convert_lora_to_gguf.py "/content/model_modified/checkpoint-1000" \
  --base "/content/model_base" \
  --outfile "/content/model_modified_quantized.gguf" \
  --outtype q8_0

assert os.path.exists("/content/model_modified_quantized.gguf")
files.download('/content/model_modified_quantized.gguf')
''';

INFO:lora-to-gguf:Loading base model: model_base
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:lora-to-gguf:Exporting model...
INFO:hf-to-gguf:blk.0.attn_q.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.0.attn_q.weight.lora_b,  torch.float32 --> F16, shape = {8, 2048}
INFO:hf-to-gguf:blk.0.attn_v.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.0.attn_v.weight.lora_b,  torch.float32 --> F16, shape = {8, 256}
INFO:hf-to-gguf:blk.1.attn_q.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.1.attn_q.weight.lora_b,  torch.float32 --> F16, shape = {8, 2048}
INFO:hf-to-gguf:blk.1.attn_v.weight.lora_a,  torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.1.attn_v.weight.lora_b,  torch.float32 --> F16, shape = {8, 256}
INFO:hf-to-gguf:blk.10.attn_q.weight.lora_a, torch.float32 --> Q8_0, shape = {2048, 8}
INFO:hf-to-gguf:blk.10.attn_q.weight.lora_b, torch.float32 --> F16, 